# Exam 2021, 8.00-13.00 for the course 1MS041 (Introduction to Data Science / Introduktion till dataanalys)

## Instructions:
1. Complete the problems by following instructions.
2. When done, submit this file with your solutions saved, following the instruction sheet.

This exam has 3 problems for a total of 40 points, to pass you need
20 points.

## Some general hints and information:
* Try to answer all questions even if you are uncertain.
* Comment your code, so that if you get the wrong answer I can understand how you thought
this can give you some points even though the code does not run.
* Follow the instruction sheet rigorously.
* This exam is partially autograded, but your code and your free text answers are manually graded anonymously.
* If there are any questions, please ask the exam guards, they will escalate it to me if necessary.
* I (Benny) will visit the exam room at around 10:30 to see if there are any questions.

## Tips for free text answers
* Be VERY clear with your reasoning, there should be zero ambiguity in what you are referring to.
* If you want to include math, you can write LaTeX in the Markdown cells, for instance `$f(x)=x^2$` will be rendered as $f(x)=x^2$ and `$$f(x) = x^2$$` will become an equation line, as follows
$$f(x) = x^2$$
Another example is `$$f_{Y \mid X}(y,x) = P(Y = y \mid X = x) = \exp(\alpha \cdot x + \beta)$$` which renders as
$$f_{Y \mid X}(y,x) = P(Y = y \mid X = x) = \exp(\alpha \cdot x + \beta)$$

## Finally some rules:
* You may not communicate with others during the exam, for example:
    * You cannot ask for help in Stack-Overflow or other such help forums during the Exam.
    * You may not communicate with AI's, for instance ChatGPT.
    * Your on-line and off-line activity is being monitored according to the examination rules.

## Good luck!

In [59]:
# Insert your anonymous exam ID as a string in the variable below
examID="XXX"


---
## Exam vB, PROBLEM 1
Maximum Points = 8


## Probability warmup
Let's say we have an exam question which consists of $20$ yes/no questions. 
From past performance of similar students, a randomly chosen student will know the correct answer to $N \sim \text{binom}(20,11/20)$ questions. Furthermore, we assume that the student will guess the answer with equal probability to each question they don't know the answer to, i.e. given $N$ we define $Z \sim \text{binom}(20-N,1/2)$ as the number of correctly guessed answers. Define $Y = N + Z$, i.e., $Y$ represents the number of total correct answers.

We are interested in setting a deterministic threshold $T$, i.e., we would pass a student at threshold $T$ if $Y \geq T$. Here $T \in \{0,1,2,\ldots,20\}$.

1. [5p] For each threshold $T$, compute the probability that the student *knows* less than $10$ correct answers given that the student passed, i.e., $N < 10$. Put the answer in `problem11_probabilities` as a list.
2. [3p] What is the smallest value of $T$ such that if $Y \geq T$ then we are 90\% certain that $N \geq 10$?

In [60]:

# Hint the PMF of N is p_N(k) where p_N is
from scipy.special import binom as binomial

p = 11/20
p_N = lambda k: float(binomial(20,k)*(1-p)**(20-k)*(p)**k)
p_Z = lambda z, n : float(binomial(20-n,z)*(0.5**(20-n)) if 0<=z<=20-n else 0.0)


In [61]:

# Part 1: 
# replace XXX to represent P(N < 10) for T = [0,1,2,...,20], i.e. your answer should be a list
# of length 21.
problem11_probabilities = []
for T in range(21):
    summation = 0.0
    density = 0.0

    for n in range(21):
        p_n = p_N(n)
        m = 20-n

        for z in range(m+1):
            p_z= p_Z(z,n)
            y = n + z
            p_nz = p_n * p_z

            if y>=T:
                density+=p_nz

                if n<10:
                    summation+=p_nz
    problem11_probabilities.append(summation/density if density > 0 else 0.0)


problem11_probabilities

[0.24928935982841186,
 0.24928935982832884,
 0.24928935982261047,
 0.24928935963549276,
 0.2492893557683933,
 0.24928929915834963,
 0.2492886751893025,
 0.24928330207958455,
 0.2492462852336602,
 0.24903902630299055,
 0.24808569900431432,
 0.24460820014975931,
 0.2349439695781523,
 0.21475641513175905,
 0.1826713919662099,
 0.14272522447072042,
 0.10227042692681909,
 0.06762809950564572,
 0.04166472439122743,
 0.024151134340423368,
 0.013287462679601604]

In [62]:

# Part 2: Give an integer between 0 and 20 which is the answer to 2.
problem12_T = 17

---
## Exam vB, PROBLEM 2
Maximum Points = 8


## Random variable generation and transformation

The purpose of this problem is to show that you can implement your own sampler, this will be built in the following three steps:

1. [2p] Implement a Linear Congruential Generator where you tested out a good combination (a large $M$ with $a,b$ satisfying the Hull-Dobell (Thm 6.8)) of parameters. Follow the instructions in the code block.
2. [2p] Using a generator construct random numbers from the uniform $[0,1]$ distribution.
3. [4p] Using a uniform $[0,1]$ random generator, generate samples from 

$$p_0(x) = \frac{\pi}{2}|\sin(2\pi x)|, \quad x \in [0,1] \enspace .$$

Using the **Accept-Reject** sampler (**Algorithm 1** in TFDS notes) with sampling density given by the uniform $[0,1]$ distribution.

In [63]:

def problem2_LCG(size=None, seed = 0):
    """
    A linear congruential generator that generates pseudo random numbers according to size.
    
    Parameters
    -------------
    size : an integer denoting how many samples should be produced
    seed : the starting point of the LCG, i.e. u0 in the notes.
    
    Returns
    -------------
    out : a list of the pseudo random numbers
    """
    
    a = 1664525
    b = 1013904223
    M = 2**32
    seed = seed
    output = []

    for _ in range(size):
        seed = ((a * seed)+b) % M
        output.append(seed/M)
    
    return output

In [64]:

def problem2_uniform(generator=None, period = 1, size=None, seed=0):
    """
    Takes a generator and produces samples from the uniform [0,1] distribution according
    to size.
    
    Parameters
    -------------
    generator : a function of type generator(size,seed) and produces the same result as problem2_LCG, i.e. pseudo random numbers in the range {0,1,...,period-1}
    period : the period of the generator
    seed : the seed to be used in the generator provided
    size : an integer denoting how many samples should be produced
    
    Returns
    --------------
    out : a list of the uniform pseudo random numbers
    """
    
    prn = generator(size=size, seed=seed)
    uniform_samples = [x / period for x in prn]
    
    return uniform_samples

In [65]:

import numpy as np

def problem2_accept_reject(uniformGenerator=None, n_iterations=None, seed=0):
    """
    Takes a generator that produces uniform pseudo random [0,1] numbers 
    and produces samples from (pi/2)*abs(sin(x*2*pi)) using an Accept-Reject
    sampler with the uniform distribution as the proposal distribution
    
    Parameters
    -------------
    generator : a function of the type generator(size,seed) that produces uniform pseudo random
    numbers from [0,1]
    seed : the seed to be used in the generator provided
    size : an integer denoting how many samples should be produced
    
    Returns
    --------------
    out : a list of the pseudo random numbers with the specified distribution
    """
    
    unif_rn= np.array(uniformGenerator(size=2 * n_iterations, seed=seed))
    
    samples = []
    for i in range(0, 2 * n_iterations, 2):
        x = unif_rn[i]        # proposed sample from g(x)
        u = unif_rn[i + 1]    # acceptance test uniform

        r = abs(np.sin(2 * np.pi * x))  # r(x) = f(x)/(M g(x))

        # accept–reject rule
        if u <= r:
            samples.append(x)

    
    return samples

---
#### Local Test for Exam vB, PROBLEM 2
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [66]:

# If you managed to solve all three parts you can test the following code to see if it runs
# you have to change the period to match your LCG though, this is marked as XXX.
# It is a very good idea to check these things using the histogram function in sagemath
# try with a larger number of samples, up to 10000 should run

print("LCG output: %s" % problem2_LCG(size=10, seed = 1))

period = 2**32

print("Uniform sampler %s" % problem2_uniform(generator=problem2_LCG, period = period, size=10, seed=1))

uniform_sampler = lambda size,seed: problem2_uniform(generator=problem2_LCG, period = period, size=size, seed=seed)

print("Accept-Reject sampler %s" % problem2_accept_reject(uniformGenerator = uniform_sampler,n_iterations=20,seed=1))

LCG output: [0.23645552527159452, 0.3692706737201661, 0.5042420323006809, 0.7048832636792213, 0.05054362863302231, 0.3695183543022722, 0.7747629624791443, 0.556188570568338, 0.0164932357147336, 0.6392460397910327]
Uniform sampler [5.505409214449919e-11, 8.597752864476435e-11, 1.1740299693790285e-10, 1.641184239832734e-10, 1.1768105587228739e-11, 8.603519627411668e-11, 1.8038855923319802e-10, 1.2949774287835183e-10, 3.840130687396415e-12, 1.488360668977333e-10]
Accept-Reject sampler [np.float64(5.505409214449919e-11), np.float64(1.1740299693790285e-10), np.float64(1.8038855923319802e-10), np.float64(5.831270047992193e-11), np.float64(1.375307788660518e-10), np.float64(5.4732877626841037e-11), np.float64(2.0044089630265327e-10), np.float64(1.5893083106077038e-10), np.float64(5.027791637884915e-11), np.float64(1.6425061072529445e-11), np.float64(3.0224932149117145e-11), np.float64(1.3571397770775068e-10), np.float64(6.122235140723631e-11), np.float64(2.1669569318181342e-10), np.float64(7.

In [67]:

# If however you did not manage to implement either part 1 or part 2 but still want to check part 3, you can run the code below

# def testUniformGenerator(size,seed):
#     set_random_seed(seed)
    
#     return [random() for s in range(size)]

# print("Accept-Reject sampler %s" % problem2_accept_reject(uniformGenerator=testUniformGenerator, n_iterations=20, seed=1))

---
## Exam vB, PROBLEM 3
Maximum Points = 8


## Concentration of measure

As you recall, we said that concentration of measure was simply the phenomenon where we expect that the probability of a large deviation of some quantity becoming smaller as we observe more samples: [0.4 points per correct answer]

1. Which of the following will exponentially concentrate, i.e. for some $C_1,C_2,C_3,C_4 $ 
$$
    P(Z - \mathbb{E}[Z] \geq \epsilon) \leq C_1 e^{-C_2 n \epsilon^2} \wedge C_3 e^{-C_4 n (\epsilon+1)} \enspace .
$$

    1. The empirical mean of i.i.d. sub-Gaussian random variables?
    2. The empirical mean of i.i.d. sub-Exponential random variables?
    3. The empirical mean of i.i.d. random variables with finite variance?
    4. The empirical variance of i.i.d. random variables with finite variance?
    5. The empirical variance of i.i.d. sub-Gaussian random variables?
    6. The empirical variance of i.i.d. sub-Exponential random variables?
    7. The empirical third moment of i.i.d. sub-Gaussian random variables?
    8. The empirical fourth moment of i.i.d. sub-Gaussian random variables?
    9. The empirical mean of i.i.d. deterministic random variables?
    10. The empirical tenth moment of i.i.d. Bernoulli random variables?

2. Which of the above will concentrate in the weaker sense, that for some $C_1$
$$
    P(Z - \mathbb{E}[Z] \geq \epsilon) \leq \frac{C_1}{n \epsilon^2}?
$$

In [68]:

# Answers to part 1, which of the alternatives exponentially concentrate, answer as a list
# i.e. [1,4,5] that is example 1, 4, and 5 concentrate
problem3_answer_1 = [1,2,9,10]

In [69]:

# Answers to part 2, which of the alternatives concentrate in the weaker sense, answer as a list
# i.e. [1,4,5] that is example 1, 4, and 5 concentrate
problem3_answer_2 = [1,2,3,5,6,7,8,9,10]

---
## Exam vB, PROBLEM 4
Maximum Points = 8


## SMS spam filtering [8p]

In the following problem we will explore SMS spam texts. The dataset is the `SMS Spam Collection Dataset` and we have provided for you a way to load the data. If you run the appropriate cell below, the result will be in the `spam_no_spam` variable. The result is a `list` of `tuples` with the first position in the tuple being the SMS text and the second being a flag `0 = not spam` and `1 = spam`.

1. [3p] Let $X$ be the random variable that represents each SMS text (an entry in the list), and let $Y$ represent whether text is spam or not i.e. $Y \in \{0,1\}$. Thus $\mathbb{P}(Y = 1)$ is the probability that we get a spam. The goal is to estimate:
$$
    \mathbb{P}(Y = 1 | \text{"free" or "prize" is in } X) \enspace .
$$
That is, the probability that the SMS is spam given that "free" or "prize" occurs in the SMS. 
Hint: it is good to remove the upper/lower case of words so that we can also find "Free" and "Prize"; this can be done with `text.lower()` if `text` a string.
2. [3p] Provide a "90\%" interval of confidence around the true probability. I.e. use the Hoeffding inequality to obtain for your estimate $\hat P$ of the above quantity. Find $l > 0$ such that the following holds:
$$
    \mathbb{P}(\hat P - l \leq \mathbb{E}[\hat P] \leq \hat P + l) \geq 0.9 \enspace .
$$
3. [2p] Repeat the two exercises above for "free" appearing twice in the SMS.

In [70]:

# Run this cell to get the SMS text data
# from exam_extras import load_sms
from Utils import load_sms
spam_no_spam = load_sms()

In [71]:
def has_free_or_prize(text: str) -> bool:
    t = text.lower()
    return ("free" in t) or ("prize" in t)


A = [(txt, y) for (txt, y) in spam_no_spam if has_free_or_prize(txt)]
nA = len(A)


# fill in the estimate for part 1 here (should be a number between 0 and 1)
problem4_hatP = (sum(y for (_, y) in A) / nA) if nA > 0 else 0.0


In [72]:

# fill in the calculated l from part 2 here
delta = 0.1
problem4_l = np.sqrt(np.log(2/delta) / (2*nA)) if nA > 0 else 0.0

In [73]:
def has_double_free(text: str) -> bool:
    t = text.lower()
    return t.count("free") >= 2

# fill in the estimate for hatP for the double free question in part 3 here (should be a number between 0 and 1)
A2 = [(txt, y) for (txt, y) in spam_no_spam if has_double_free(txt)]
nA2 = len(A2)

problem4_hatP2 = (sum(y for (_, y) in A2) / nA2) if nA2 > 0 else 0.0

In [74]:

# fill in the estimate for l for the double free question in part 3 here
problem4_l2 = np.sqrt(np.log(2/delta) / (2*nA2)) if nA2 > 0 else 0.0

---
## Exam vB, PROBLEM 5
Maximum Points = 8


## Markovian travel

The dataset `Travel Dataset - Datathon 2019` is a simulated dataset designed to mimic real corporate travel systems -- focusing on flights and hotels. The file is at `data/flights.csv` in the same folder as `Exam.ipynb`, i.e. you can use the path `data/flights.csv` from the notebook to access the file.

1. [2p] In the first code-box 
    1. Load the csv from file `data/flights.csv`
    2. Fill in the value of the variables as specified by their names.
2. [2p] In the second code-box your goal is to estimate a Markov chain transition matrix for the travels of these users. For example, if we enumerate the cities according to alphabetical order, the first city `'Aracaju (SE)'` would correspond to $0$. Each row of the file corresponds to one flight, i.e. it has a starting city and an ending city. We model this as a stationary Markov chain, i.e. each user's travel trajectory is a realization of the Markov chain, $X_t$. Here, $X_t$ is the current city the user is at, at step $t$, and $X_{t+1}$ is the city the user travels to at the next time step. This means that to each row in the file there is a corresponding pair $(X_{t},X_{t+1})$. The stationarity assumption gives that for all $t$ there is a transition density $p$ such that $P(X_{t+1} = y | X_t = x) = p(x,y)$ (for all $x,y$). The transition matrix should be `n_cities` x `n_citites` in size.
3. [2p] Use the transition matrix to compute out the stationary distribution.
4. [2p] Given that we start in 'Aracaju (SE)' what is the probability that after 3 steps we will be back in 'Aracaju (SE)'?

In [75]:
import pandas as pd

df = pd.read_csv('data/flights.csv')
df.head()




,travelCode,userCode,from,to,flightType,price,time,distance,agency,date
0,0,0,Recife (PE),Florianopolis (SC),firstClass,1434.38,1.76,676.53,FlyingDrops,09/26/2019
1,0,0,Florianopolis (SC),Recife (PE),firstClass,1292.29,1.76,676.53,FlyingDrops,09/30/2019
2,1,0,Brasilia (DF),Florianopolis (SC),firstClass,1487.52,1.66,637.56,CloudFy,10/03/2019
3,1,0,Florianopolis (SC),Brasilia (DF),firstClass,1127.36,1.66,637.56,CloudFy,10/04/2019
4,2,0,Aracaju (SE),Salvador (BH),firstClass,1684.05,2.16,830.86,CloudFy,10/10/2019


In [76]:
# df['from'].unique() # displays the unique cities
df['to'].nunique() # retuns the number of unique cities

9

In [77]:
# df['userCode'].value_counts()  # value_counts returns the frequency of each unique usercode

df['userCode'].nunique()  # nunique returns how many unique usercodes exist


1335

In [78]:
len(df)


271888

In [79]:

number_of_cities = df['to'].nunique()
number_of_userCodes = df['userCode'].nunique()
number_of_observations = len(df)

In [80]:

# This is a very useful function that you can use for part 2. You have seen this before when parsing the
# pride and prejudice book.

def makeFreqDict(myDataList):
    '''Make a frequency mapping out of a list of data.

    Param myDataList, a list of data.
    Return a dictionary mapping each unique data value to its frequency count.'''

    freqDict = {} # start with an empty dictionary

    for res in myDataList:
        if res in freqDict: # the data value already exists as a key
                freqDict[res] = freqDict[res] + 1 # add 1 to the count using sage integers
        else: # the data value does not exist as a key value
            freqDict[res] = 1 # add a new key-value pair for this new data value, frequency 1

    return freqDict # return the dictionary created

In [81]:

cities = set(df['from']).union(set(df['to']))

unique_cities = sorted(set(cities)) # The unique cities
n_cities = len(unique_cities) # The number of unique citites

# Count the different transitions
transitions = list(zip(df['from'], df['to'])) # A list containing tuples ex: ('Aracaju (SE)','Rio de Janeiro (RJ)') of all transitions in the text

transition_counts = makeFreqDict(transitions) # A dictionary that counts the number of each transition 
# ex: ('Aracaju (SE)','Rio de Janeiro (RJ)'):4



indexToCity = {i: city for i, city in enumerate(unique_cities)} # A dictionary that maps the n-1 number to the n:th unique_city,
# ex: 0:'Aracaju (SE)'
cityToIndex = {city: i for i, city in enumerate(unique_cities)} # The inverse function of indexToWord, 
# ex: 'Aracaju (SE)':0

# Part 3, finding the maximum likelihood estimate of the transition matrix


# The transition matrix should be ordered in such a way that
# p_{'Aracaju (SE)','Rio de Janeiro (RJ)'} = transition_matrix[cityToIndex['Aracaju (SE)'],cityToIndex['Rio de Janeiro (RJ)']]
# and represents the probability of travelling Aracaju (SE)->Rio de Janeiro (RJ)

# Make sure that the transition_matrix does not contain np.nan from division by zero for instance

transition_matrix = np.zeros((n_cities,n_cities)) # a numpy array of size (n_cities,n_cities)

for (city_from,city_to), count in transition_counts.items():
    i = cityToIndex[city_from]
    j = cityToIndex[city_to]
    transition_matrix[i,j]+= count

row_sums = transition_matrix.sum(axis=1)

for i in range(n_cities):
    if row_sums[i] > 0:
        transition_matrix[i, :] /= row_sums[i]


transition_matrix[
    cityToIndex['Aracaju (SE)'],
    cityToIndex['Rio de Janeiro (RJ)']
]

np.float64(0.07570384698044272)

In [82]:

# This should be a numpy array of length n_cities which sums to 1 and is all positive

P = transition_matrix
n = P.shape[0]

# start from uniform distribution
pi = np.ones(n) / n

# iterate until convergence
for _ in range(10000):
    pi_next = pi @ P
    if np.linalg.norm(pi_next - pi, ord=1) < 1e-12:
        break
    pi = pi_next

# ensure proper normalization (numerical safety)
pi = pi / pi.sum()

stationary_distribution_problem5 = pi


In [83]:

# Compute the return probability for part 3 of problem 5


i = cityToIndex['Aracaju (SE)']

P3 = np.linalg.matrix_power(transition_matrix, 3)
return_probability_problem5 = P3[i, i]


---
#### Local Test for Exam vB, PROBLEM 5
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [84]:
# Once you have created all your functions, you can make a small test here to see
# what would be generated from your model.
import numpy as np

start = np.zeros(shape=(n_cities,1))
start[cityToIndex['Aracaju (SE)'],0] = 1

current_pos = start
for i in range(10):
    random_word_index = np.random.choice(range(n_cities),p=current_pos.reshape(-1))
    current_pos = np.zeros_like(start)
    current_pos[random_word_index] = 1
    print(indexToCity[random_word_index],end='->')
    current_pos = (current_pos.T@transition_matrix).T

Aracaju (SE)->Natal (RN)->Campo Grande (MS)->Florianopolis (SC)->Campo Grande (MS)->Recife (PE)->Brasilia (DF)->Recife (PE)->Rio de Janeiro (RJ)->Brasilia (DF)->

---
## Exam vB, PROBLEM 6
Maximum Points = 8


## Black box testing

In the following problem we will continue with our SMS spam / nospam data. This time we will try to approach the problem as a pattern recognition problem. For this particular problem I have provided you with everything -- data is prepared, split into train-test sets and a black-box model has been fitted on the training data and predicted on the test data. Your goal is to calculate test metrics and provide guarantees for each metric.

1. [2p] Compute precision for class 1 (see notes 8.3.2 for definition), then provide an interval using Hoeffding's inequality for a 95\% confidence.
2. [2p] Compute recall for class 1(see notes 8.3.2 for definition), then provide an interval using Hoeffding's inequality for a 95\% interval.
3. [2p] Compute accuracy (0-1 loss), then provide an interval using Hoeffding's inequality for a 95\% interval.
4. [2p] If we would have used a classifier with VC-dimension 3, would we have obtained a smaller interval for accuracy by using all data?

In [85]:

# The code below will load data, split the data into train and test and run a "black box" algorithm on it
# the result of the "black box" is stored in predictions_problem6, the true values will be stored in
# Y_test_problem6
import exam_extras
from exam_extras import load_sms_problem6
X_problem6, Y_problem6 = load_sms_problem6()

X_train_problem6,X_test_problem6,Y_train_problem6,Y_test_problem6 = exam_extras.train_test_split(X_problem6,Y_problem6)
predictions_problem6 = exam_extras.knn_predictions(X_train_problem6,Y_train_problem6,X_test_problem6,k=4)

ModuleNotFoundError: No module named 'exam_extras'

In [ ]:

# Compute the precision of predictions_problem6 with respect to Y_test_problem6
problem6_precision = XXX

In [ ]:

# Compute the interval length l of precision of predictions_problem6 with respect to Y_test_problem6, with the same definition of l as in problem 4
problem6_precision_l = XXX

In [ ]:

# Repeat the same procedure but for recall
problem6_recall = XXX

In [ ]:

problem6_recall_l = XXX

In [ ]:

# Repeat the same procedure but for accuracy or 0-1 loss
problem6_accuracy = XXX

In [ ]:

problem6_accuracy_l = XXX

In [ ]:

# Below you will calculate the interval parameter l for a classifier running on all data with a VC dimension of 3
# put the value in problem6_VC_l and answer problem_VC_smaller as True if the interval is smaller than the test-accuracy above
# if not answer False. Make sure you replace XXX with something even if you only answer one of them.
problem6_VC_l = XXX # number
problem6_VC_smaller = XXX #True / False